In [ ]:
#!pip install --force crewai

# Funtions

In [ ]:
# Bibliotecas do generabb
from generabb.llm_framework.api_models import CallAPIModels
from changes.api_embeddings import CallAPIEmbeddings
# Importa funções para criação do Contexto de Serviço
from generabb.llm_framework.service_context import *
# Criação do contexto de armazenamento
from llama_index import (load_index_from_storage,
                         StorageContext,
                         VectorStoreIndex)
from llama_index.bridge.pydantic import BaseModel, Field
# Leitura de arquivos texto
from llama_index.readers.file.base import SimpleDirectoryReader
from llama_index.retrievers import BaseRetriever
from llama_index.schema  import BaseNode
from typing import List
from glob import glob
from langchain.tools import Tool


def get_context_service(max_tokens=4000,
                        max_new_tokens=4000,
                        chunk_overlap=100,
                        as_global=False):
    
    global llm, tokenizer, embed_model, NODE_PARSER, service_context

    llm, tokenizer = CallAPIModels(config_path='/home/wsl/projects/frameworks/crewai/config.env')

    embed_model = CallAPIEmbeddings(config_path='/home/wsl/projects/frameworks/crewai/config.env')

    # Configuração para uso de recursos via API
    NODE_PARSER, service_context = DefineServiceContext(llm=llm, 
                                                        tokenizer=tokenizer, 
                                                        embed_model=embed_model,
                                                        max_tokens=max_tokens,
                                                        max_new_tokens=max_new_tokens,
                                                        chunk_overlap=chunk_overlap,
                                                        as_global=as_global)


def load_index(storage_path, service_context):
    """
    Função para realizar a leitura de um diretório com nome id_idx.

    Entradas:
        - storage_path: Raiz onde se encontram os arquivos do index
        - service_context: Contexto de serviço a ser utilizado
    Saída:
        - index_tmp: Índice carregado
    """

    # doc_txt_files_path
    index_tmp = load_index_from_storage(
        storage_context=StorageContext.from_defaults(
            persist_dir=storage_path),
        service_context=service_context)
    return index_tmp


def doc_to_nodes(file: str,
                  node_parser: SentenceSplitter,
                  ) -> List[BaseNode]:
    # Leitura do documento
    documents = SimpleDirectoryReader(
                                input_files=[file],
                                filename_as_id=True
                                ).load_data()

    # Criar os Nodes
    nodes = node_parser.get_nodes_from_documents(documents)

    return nodes


def create_vector_index(file: str,
                        node_parser: SentenceSplitter,
                        service_context: ServiceContext,
                        show_progress=True
                        ) -> VectorStoreIndex:
    
    nodes = doc_to_nodes(file,
                          node_parser)
    
    # Create index
    v_index = VectorStoreIndex(nodes=nodes,
                               service_context=service_context,
                               show_progress=show_progress)

    return v_index


def create_index_from_path(files_path: str,
                           node_parser: SentenceSplitter,
                           service_context: ServiceContext):
    
    files_list = glob(files_path + "/*")
    files_list = [file for file in files_list if 'skip' not in file]

    file = files_list[0]
    print("Adicionando arquivo", file)
    v_index = create_vector_index(file, node_parser, service_context)

    if len(files_list) > 1:
        for file in files_list[1:]:
            print("Adicionando arquivo", file)
            insert_doc(file, node_parser, v_index)

    print('Arquivos adicionados ao Index com sucesso!')

    return v_index


def insert_doc(file: str, 
                node_parser: SentenceSplitter,
                v_index: VectorStoreIndex):
    
    # Leitura do documento
    documents = SimpleDirectoryReader(
                                    input_files=[file],
                                    filename_as_id=True
                                    ).load_data()
    # Criar os Nodes
    nodes = node_parser.get_nodes_from_documents(documents)
    
    # Inserir na VectorStore
    v_index.insert_nodes(nodes)


def update_doc(file: str,
                v_index: VectorStoreIndex,
                delete_from_docstore=True):
    # Leitura do documento
    documents = SimpleDirectoryReader(
                                input_files=[file],
                                filename_as_id=True
                                ).load_data()

    for doc in documents:
        # Inserir na VectorStore
        v_index.update_ref_doc(
            document=doc,
            update_kwargs={"delete_kwargs": {"delete_from_docstore": True}}
        )


def delete_doc(file_name: str, 
               v_index: VectorStoreIndex,
               delete_from_docstore = True):

    v_index.delete_ref_doc(ref_doc_id=file_name,
                           delete_from_docstore=delete_from_docstore)


class RetrieverInput(BaseModel):
    """Input to the retriever."""

    query: str = Field(description="query to look up in retriever")


def create_li_retriever_tool(
    retriever: BaseRetriever, name: str, description: str
) -> Tool:
    """Create a tool to do retrieval of documents.

    Args:
        retriever: The retriever to use for the retrieval
        name: The name for the tool. This will be passed to the language model,
            so should be unique and somewhat descriptive.
        description: The description for the tool. This will be passed to the language
            model, so should be descriptive.

    Returns:
        Tool class to pass to an agent
    """
    return Tool(
        name=name,
        description=description,
        func=retriever.retrieve,
        coroutine=retriever.aretrieve,
        args_schema=RetrieverInput,
    )



# Initialize Models

In [2]:
get_context_service()

[nltk_data] Error loading punkt: HTTP Error 403: Template-
[nltk_data]     BloqueioDefaultProxyTransparente


Contexto de Serviço criado com sucesso!


# LlamaIndex

## New Index

In [34]:
files_path = "/home/wsl/projects/frameworks/crewai/data/cartao_pf"
pf_index = create_index_from_path(files_path, NODE_PARSER, service_context)

Adicionando arquivo /home/wsl/projects/frameworks/crewai/data/cartao_pf/718-2.txt


Generating embeddings: 100%|██████████| 7/7 [00:09<00:00,  1.39s/it]


Adicionando arquivo /home/wsl/projects/frameworks/crewai/data/cartao_pf/395-2.txt
Adicionando arquivo /home/wsl/projects/frameworks/crewai/data/cartao_pf/718-1.txt
Aguardando 24 segundos para liberação da API.
Aguardando 6 segundos para liberação da API.
Adicionando arquivo /home/wsl/projects/frameworks/crewai/data/cartao_pf/395-1.txt
Aguardando 29 segundos para liberação da API.
Aguardando 6 segundos para liberação da API.
Arquivos adicionados ao Index com sucesso!


In [35]:
pf_index.ref_doc_info

{'/home/wsl/projects/frameworks/crewai/data/cartao_pf/718-2.txt': RefDocInfo(node_ids=['408b08be-51a4-41b6-a374-972dc3630c71', '3b7183d3-a3c9-449c-9e85-0215b0c8a2f0', '18854b9f-d44b-43fc-a164-8202ad7b5c52', 'd88b0cff-b8b4-46d5-817d-b45c325d4f34', 'a03117d7-d60c-47a3-a647-678976f61bcf', '9a251efc-feb1-4e05-8cd5-128a07aa51fa', '78ff0006-c8a6-4113-b679-390c5f13b8a9'], metadata={}),
 '/home/wsl/projects/frameworks/crewai/data/cartao_pf/395-2.txt': RefDocInfo(node_ids=['edb5bb25-f2c9-4fd3-b73a-a5ac483423de', 'cef6bea5-1557-476c-aa9b-265e135c33c7', '8d4f78e6-5eb2-4b2c-894c-1406ad0c69ad', 'f762b0e5-8b97-47a4-87d9-af3f82a51815', '5cacbbba-7418-42f2-a5aa-0d7e8bbdb696', 'aa45ea21-e071-44a8-b0c5-30ef5a5a6e3f', '55e60c3c-0937-4f98-a04e-0921e735443f', '15a03b20-abc4-4980-84cb-ce1675115b73', '669d9299-4e5e-447e-ab2f-bc7e57f8053f', 'e80cde26-a94b-43c9-a3cd-b680a557d283', '1c741f71-2a72-452e-a369-730da8dbf52f', 'd7d1566c-36f1-4cd1-b216-4854685d4eb6', 'da04c633-2013-4918-b2a6-6f92be319a39', '9cc6aef7-1

In [37]:
files_path = "/home/wsl/projects/frameworks/crewai/data/cartao_pj"
pj_index = create_index_from_path(files_path, NODE_PARSER, service_context)

Adicionando arquivo /home/wsl/projects/frameworks/crewai/data/cartao_pj/392-IA.txt


Generating embeddings: 100%|██████████| 2/2 [00:01<00:00,  1.40it/s]


Adicionando arquivo /home/wsl/projects/frameworks/crewai/data/cartao_pj/392-2.txt
Adicionando arquivo /home/wsl/projects/frameworks/crewai/data/cartao_pj/395-2.txt
Aguardando 25 segundos para liberação da API.
Aguardando 8 segundos para liberação da API.
Adicionando arquivo /home/wsl/projects/frameworks/crewai/data/cartao_pj/395-1.txt
Adicionando arquivo /home/wsl/projects/frameworks/crewai/data/cartao_pj/392-1.txt
Aguardando 24 segundos para liberação da API.
Aguardando 7 segundos para liberação da API.
Arquivos adicionados ao Index com sucesso!


In [38]:
pj_index.ref_doc_info

{'/home/wsl/projects/frameworks/crewai/data/cartao_pj/392-IA.txt': RefDocInfo(node_ids=['3c9e0ee7-a922-4e5c-9409-e9b9317b7f4e', '5cfdbbf3-42d7-4e42-9c9c-ed0206497e34'], metadata={}),
 '/home/wsl/projects/frameworks/crewai/data/cartao_pj/392-2.txt': RefDocInfo(node_ids=['0f14920e-9941-4d88-9d0e-13c99378cc53', 'fc770444-3839-4204-92b0-e533727c8843', '7165d74b-9a14-48fd-bb6e-5b6c12a6a85a', '6fa4d65e-1948-4694-b3d3-6d2968d214dc', 'fa949c9f-4840-4e0a-aa7b-82a35351719c', 'da49cac8-eff8-4739-ac7f-3f125cbb846b', '7edca88b-e9a2-4446-9300-c0bb747b6216', '8babe75b-f5fa-4345-b43f-51fe64f654b0', '02a0a9b4-3b4f-448e-8b41-1a3c103905f1', 'ca49c368-98d7-419b-8721-d7bab6f7e43e', 'cb7c751b-63dc-4c03-aa18-65cf4feccc57', 'aea29138-1305-4b67-8be5-d1999478dbdb', 'fb9d1355-2db1-43c5-98cd-d4a113f54542', '7f6fe279-189c-4686-ac2a-c5b1a247b126', '5eea5edd-5045-400e-aaea-bfab1db91955', '405faee4-ad04-4d8d-89ca-7e80b4ec7403'], metadata={}),
 '/home/wsl/projects/frameworks/crewai/data/cartao_pj/395-2.txt': RefDocInf

In [40]:
# Save it local
pf_index.storage_context.persist(persist_dir="./tmp/pf_index")
pj_index.storage_context.persist(persist_dir="./tmp/pj_index")

## Load Index

In [4]:
pf_index = load_index("./tmp/pf_index/", service_context)
pj_index = load_index("./tmp/pj_index/", service_context)

In [6]:
pf_index.ref_doc_info.keys()

dict_keys(['/home/wsl/projects/frameworks/crewai/data/cartao_pf/718-2.txt', '/home/wsl/projects/frameworks/crewai/data/cartao_pf/395-2.txt', '/home/wsl/projects/frameworks/crewai/data/cartao_pf/718-1.txt', '/home/wsl/projects/frameworks/crewai/data/cartao_pf/395-1.txt'])

In [7]:
pj_index.ref_doc_info.keys()

dict_keys(['/home/wsl/projects/frameworks/crewai/data/cartao_pj/392-IA.txt', '/home/wsl/projects/frameworks/crewai/data/cartao_pj/392-2.txt', '/home/wsl/projects/frameworks/crewai/data/cartao_pj/395-2.txt', '/home/wsl/projects/frameworks/crewai/data/cartao_pj/395-1.txt', '/home/wsl/projects/frameworks/crewai/data/cartao_pj/392-1.txt'])

# CrewAI

In [29]:
# Install CrewAI
# !pip install crewai

# Install duckduckgo-search for this example:
# !pip install -U duckduckgo-search

import os
from crewai import Agent, Task, Crew, Process

from langchain.tools import DuckDuckGoSearchRun
#from langchain.tools.retriever import create_retriever_tool
#from langchain_community.document_loaders import TextLoader

get_context_service()
search_tool = DuckDuckGoSearchRun()
retriver_pj = pj_index.as_retriever(similarity_top_k=1)
retriver_pf = pf_index.as_retriever(similarity_top_k=1)

retriever_pj_tool = create_li_retriever_tool(
    retriever=retriver_pj,
    name="Informações Cartão Pessoa Jurídica",
    description="Pesquisa e retorna itens referentes a cartões de crédito para pessoa jurídica."
)


retriever_pf_tool = create_li_retriever_tool(
    retriever=retriver_pf,
    name="Informações Cartão Pessoa Física",
    description="Pesquisa e retorna itens referentes a cartões de crédito para pessoa física."
)

# Define your agents with roles and goals
researcher_pj = Agent(
  role='Analista Senior Cartão Pessoa Jurídica',
  goal='Compreender documentos de referência sobre cartões para pessoa jurídica e orientar atendimentos',
  backstory="""Você trabalha a muitos anos como Analista de Cartões para Pessoas Jurídicas.
  É uma referência no assunto, todos buscam em você orientações sobre o assunto Cartões para Pessoas Jurídicas.
  Você tem habilidade de explicar com didática e facilidade para as pessoas assuntos complicados no tema.""",
  verbose=True,
  allow_delegation=False,
  tools=[retriever_pj_tool],
  llm=llm
)

# Define your agents with roles and goals
researcher_pf = Agent(
  role='Analista Senior Cartão Pessoa Física',
  goal='Compreender documentos de referência sobre cartões para Pessoas Físicas e orientar atendimentos',
  backstory="""Você trabalha a muitos anos como Analista de Cartões para Pessoas Físicas.
  É uma referência no assunto, todos buscam em você orientações sobre o assunto Cartões para Pessoas Físicas.
  Você tem habilidade de explicar com didática e facilidade para as pessoas assuntos complicados no tema.
  Você sempre pesquisa na documentação disponível as informações solicitadas.""",
  verbose=True,
  allow_delegation=False,
  tools=[retriever_pf_tool],
  llm=llm
)

# Define your agents with roles and goals
cx = Agent(
  role='Atendente de Consumidores',
  goal='Atender os pedidos e responder as dúvidas dos clientes buscando apoio de Analista Seniores nas respostas.',
  backstory="""Você é um renomado atendente de consumidores.
  Você possui uma habilidade de conseguir traduzir assuntos complexos em temas simples.
  Você sempre é didático e busca apoio nos Analistas Seniores de acordo com o tema de conhecimento de cada um.
  Você sempre pesquisa na documentação disponível as informações solicitadas.""",
  verbose=True,
  allow_delegation=True,
  llm=llm
)

# Create tasks for your agents
task11 = Task(
  description="""Conduza uma análise sobre a documentação existente de acordo com a pergunta realizada.
  Sua resposta final deve ser uma orientação de forma didática a explicar a resposta para a questão.
  Você sempre informa sua referência. Quando o tema é Cartão Pessoa Física você direciona ao analista de Cartão para Pessoa Física.""",
  agent=researcher_pf
)
task12 = Task(
  description="""Conduza uma análise sobre a documentação existente de acordo com a pergunta realizada.
  Sua resposta final deve ser uma orientação de forma didática a explicar a resposta para a questão.
  Você sempre informa sua referência. Quando o tema é Cartão Pessoa Jurídica você direciona ao analista de Cartão para Pessoa Jurídica.""",
  agent=researcher_pj
)
task2_pf = Task(
  description="""Quais são os tipos de Cartão de Crédito para Pessoa Física?""",
  agent=cx
)

# Instantiate your crew with a sequential process
crew = Crew(
  agents=[cx, researcher_pj, researcher_pf],
  tasks=[task2_pf],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

[nltk_data] Error loading punkt: HTTP Error 403: Template-
[nltk_data]     BloqueioDefaultProxyTransparente


Contexto de Serviço criado com sucesso!

[DEBUG]: Working Agent: Atendente de Consumidores

[INFO]: Starting Task: Quais são os tipos de Cartão de Crédito para Pessoa Física?


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Ask question to co-worker
Action Input: Analista Senior Cartão Pessoa Física|Quais são os tipos de Cartão de Crédito para Pessoa Física?|Estou tentando responder uma pergunta do cliente sobre os diferentes tipos de cartões de crédito disponíveis para pessoas físicas.

> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Informações Cartão Pessoa Física
Action Input: Tipos de cartões de crédito para pessoa física.[NodeWithScore(node=TextNode(id_='e7f927ec-5673-450c-9acc-2ec1209ea1fa', embedding=None, metadata={}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type

In [30]:
task2_pj = Task(
  description="""Quais são os tipos de Cartão de Crédito para Pessoa Jurídica?""",
  agent=cx
)

# Instantiate your crew with a sequential process
crew = Crew(
  agents=[cx, researcher_pj, researcher_pf],
  tasks=[task2_pj],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)


[DEBUG]: Working Agent: Atendente de Consumidores

[INFO]: Starting Task: Quais são os tipos de Cartão de Crédito para Pessoa Jurídica?


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Ask question to co-worker
Action Input: Analista Senior Cartão Pessoa Jurídica|Quais são os tipos de Cartão de Crédito para Pessoa Jurídica?|Estou tentando responder uma pergunta do cliente sobre os diferentes tipos de cartões de crédito disponíveis para pessoas jurídicas. Preciso desta informação para fornecer uma resposta completa e precisa.

> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Informações Cartão Pessoa Jurídica
Action Input: Tipos de cartões de crédito para pessoa jurídica.[NodeWithScore(node=TextNode(id_='275a0078-aa93-4e6f-b78f-fb5ecda6a34e', embedding=None, metadata={}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], exclude